### 코로나가 서울시 대중교통에 미친영향

- 버스 vs 지하철 vs 공공자전거 증감률 비교
- 코로나 연령별/성별, 감염자 수/사망자수/치명율 현황 (월별)

### * 작업순서
 * 버스 /지하철/공공자전거 이용 증감률 계산
 * 연도별/월별 버스/지하철/공공자전거 이용 인원 합계
 * 코로나 전,후 월별  버스/지하철/공공자전거 이용 증감률
 * 2019년 대비 2020/2021(7월)년 증감율 구하기
 * 거리두기 단계별 버스/지하철/공공자전거 증감율 계산

In [3]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

# plotly
import plotly as pl

import os 
import folium 

In [2]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [6]:
# pip install cufflinks
os.getcwd()

'c:\\Users\\JSSON\\JS_Python\\02.team_project_data\\01.source'

In [6]:
# 불러올 폴더 위치 지정
dt_path = "C:/Users/NaEunSu/critical_shoot_project/01.source/01.데이터가공/02.데이터/"

# 버스 이용현황 불러오기
pd.read_csv(dt_path + "월별 버스이용객수.csv",encoding="cp949"  )

,year,승하객,하차객
0,201901,128937436,132780532
1,201902,124789738,128547850
2,201903,140433878,144709644
3,201904,143084559,147452704
4,201905,139996976,144202984
5,201906,138784006,142926564
6,201907,140962485,145053974
7,201908,133809814,137576840
8,201909,134382556,138184213
9,201910,138805578,142636150


In [8]:
# 데이터 생성 및 검증 
# 해당 위치의 모든 파일명을 list 형태로 가져옴 
fd_filenm = os.listdir("C:/Users/NaEunSu/critical_shoot_project/01.source/01.데이터가공/02.데이터/")

# 확장자가 csv인 경우에만 리스트에 담도록 하는 구문
fd_filenm=[filenm for filenm in fd_filenm if filenm[-3:]=='csv']
fd_filenm

['metro_total_v01_210812.csv',
 'people_data5.csv',
 '버스_이용자현황_2019_2021.csv',
 '서울 공공 자전거 대여소별 이용건수(월별).csv',
 '서울버스정류소좌표.csv',
 '서울지하철역좌표.csv',
 '월별 버스이용객수.csv']

In [9]:

def mergeReadCsv(arg_fnm_list):
# 버스 이용현황 데이터 폴더
    fd_path = '../02.data/서울시버스_이용현황/'
    df_column = ['사용일자', '노선번호', '역명', '승차총승객수', '하차총승객수', '등록일자']

    err_li=[]
    dfs = pd.DataFrame()

    for fnm in arg_fnm_list:
        print(fnm)
        # f=open(fd_path+fnm, "r")
        # print(type(f), f)
        # print()
        # f.close()

        # try 문을 사용한 것은 read_csv 로 파일을 열었을때 파일의 정보는 cp949 인코딩으로 나와 있지만 실제 열리지 않는 이슈가 있음.
        # 노트패드로 해당 파일을 열었을때, 컬럼 열은 6열 데이터의 열은 7열로 되어 있었으며 "" 빈값이 작성되어 있었음.
        try:
            df = pd.read_csv(fd_path+fnm, encoding='cp949', skiprows=1)
        # 오류가 발생된 경우 
        except:
            print("오류발생:", fnm)
            err_li.append(fnm)
            df = pd.read_csv(fd_path+fnm, encoding='utf-8', skiprows=1)
            if df.shape[1] == 7:
                df = df.iloc[:, :-1]
            df.columns = df_column            
            dfs = pd.concat([dfs, df])
        # 정상적으로 read_csv 가 수행된 경우 
        else:
            # 파일을 합치는 중에 컬럼 열은 6열, 데이터는 7열인 파일이 있어서 변경하기 위해 반영
            if df.shape[1] == 7:
                df = df.iloc[:, :-1]
            df.columns = df_column
            dfs = pd.concat([dfs, df])
    print(err_li)
    return dfs    


In [11]:
tot_df=mergeReadCsv(fd_filenm)

BUS_STATION_BOARDING_MONTH_201901.csv
BUS_STATION_BOARDING_MONTH_201902.csv
BUS_STATION_BOARDING_MONTH_201903.csv
BUS_STATION_BOARDING_MONTH_201904.csv
BUS_STATION_BOARDING_MONTH_201905.csv
BUS_STATION_BOARDING_MONTH_201906.csv
BUS_STATION_BOARDING_MONTH_201907.csv
BUS_STATION_BOARDING_MONTH_201908.csv
BUS_STATION_BOARDING_MONTH_201909.csv
BUS_STATION_BOARDING_MONTH_201910.csv
BUS_STATION_BOARDING_MONTH_201911.csv
BUS_STATION_BOARDING_MONTH_201912.csv
BUS_STATION_BOARDING_MONTH_202001.csv
BUS_STATION_BOARDING_MONTH_202002.csv
BUS_STATION_BOARDING_MONTH_202003.csv
BUS_STATION_BOARDING_MONTH_202004.csv
BUS_STATION_BOARDING_MONTH_202005.csv
BUS_STATION_BOARDING_MONTH_202006.csv
BUS_STATION_BOARDING_MONTH_202007.csv
BUS_STATION_BOARDING_MONTH_202008.csv
BUS_STATION_BOARDING_MONTH_202009.csv
BUS_STATION_BOARDING_MONTH_202010.csv
BUS_STATION_BOARDING_MONTH_202011.csv
BUS_STATION_BOARDING_MONTH_202012.csv
BUS_STATION_BOARDING_MONTH_202101.csv
BUS_STATION_BOARDING_MONTH_202102.csv
BUS_STATION_

In [12]:
tot_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32505794 entries, 0 to 1048573
Data columns (total 6 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   사용일자    int64 
 1   노선번호    object
 2   역명      object
 3   승차총승객수  int64 
 4   하차총승객수  int64 
 5   등록일자    int64 
dtypes: int64(4), object(2)
memory usage: 1.7+ GB


In [14]:
tot_df=tot_df.drop('등록일자', axis=1)

In [ ]:
tot_df.dtypes()

In [16]:
modi_df = tot_df.astype({'사용일자': 'str'})

In [18]:
modi_df.dtypes

사용일자      object
노선번호      object
역명        object
승차총승객수     int64
하차총승객수     int64
dtype: object

In [19]:
# 특정컬러명 변경 inplace True 는 해당 dataframe 에 반영여부
modi_df.rename(columns={'사용일자': '사용월'}, inplace=True)

In [20]:
modi_df.columns

Index(['사용월', '노선번호', '역명', '승차총승객수', '하차총승객수'], dtype='object')

In [21]:
modi_df=modi_df.groupby(['사용월', '노선번호', '역명']).sum(['승차총승객수', '하차총승객수'])

In [23]:
modi_df.reset_index(inplace=True)

In [26]:
# 파일저장
modi_df.to_csv('버스_이용자현황_2019_2021.csv', sep=',' ,encoding='cp949')